### MACHINE UNLEARNING

## Import Dependencies

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import variational
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
from itertools import cycle
import os
import time
import math
import pandas as pd
from collections import OrderedDict
from sklearn.linear_model import LogisticRegression
import copy
import torch.nn as nn
from torch.autograd import Variable
from typing import List
import itertools
from tqdm.autonotebook import tqdm
from models import *
import models
from logger import *
import wandb
from thirdparty.repdistiller.helper.util import adjust_learning_rate as sgda_adjust_learning_rate
from thirdparty.repdistiller.distiller_zoo import DistillKL, HintLoss, Attention, Similarity, Correlation, VIDLoss, RKDLoss
from thirdparty.repdistiller.distiller_zoo import PKT, ABLoss, FactorTransfer, KDSVD, FSP, NSTLoss
from thirdparty.repdistiller.helper.loops import train_distill, train_distill_hide, train_distill_linear, train_vanilla, train_negrad, train_bcu, train_bcu_distill, validate
from thirdparty.repdistiller.helper.pretrain import init
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = ""

/var/folders/mw/2gycpkq57p97cpwvmwvm69m00000gn/T/ipykernel_3251/2126072653.py:20: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Train Original Model

This cell will run main.py which generate a model and leave many checkpoints for later references. We
prepare standard_model_for_pilot for the sake of pilot.

dataset: mnist
model: mlp
dataroot: data/MNIST
resume from standard_model_for_pilot.pt in checkpoints folder

In [2]:
%run main.py --dataset mnist --model mlp --dataroot=data/MNIST/ --filters 1.0 --lr 0.001 \
--resume checkpoints/standard_model_for_pilot.pt --disable-bn \
--weight-decay 0.1 --batch-size 128 --epochs 31 --seed 3

Checkpoint name: mnist_mlp_1_0_forget_None_lr_0_001_bs_128_ls_ce_wd_0_1_seed_3
[Logging in mnist_mlp_1_0_forget_None_lr_0_001_bs_128_ls_ce_wd_0_1_seed_3_training]
confuse mode: False
split mode: None
Number of Classes: 10
State OrderedDict([('layers.0.weight', tensor([[-0.0832, -0.0317,  0.0080,  ...,  0.0650, -0.0346, -0.0338],
        [-0.0027, -0.0888, -0.0519,  ..., -0.0379,  0.0615,  0.0149],
        [-0.1416, -0.0336,  0.0156,  ...,  0.0427,  0.0694,  0.0308],
        ...,
        [-0.0182, -0.0285,  0.0117,  ...,  0.0094, -0.0225, -0.0526],
        [-0.0159,  0.0288, -0.0066,  ...,  0.0381,  0.0633,  0.0451],
        [ 0.0131, -0.0608,  0.0516,  ..., -0.0093,  0.0415, -0.0130]])), ('layers.0.bias', tensor([ 0.2570,  0.5360, -0.9828, -0.2736, -0.2282,  0.3082,  0.4359, -0.1057,
        -0.0857,  0.2738,  0.0112, -0.4476,  0.2455,  0.2458, -0.0411, -0.3458,
        -0.4463,  0.1637, -0.5212,  0.0843,  0.1513, -0.1780, -0.6362,  0.2063,
         0.4134,  0.2404, -0.3497, -0.2096, -

# Note
After running this cell, the code will generate a trace of checkpoints under the "checkpoints" folder.
They are useful for later cell codes. But you could remove them after completing the experiment.

Also, the code would also download a MNIST dataset in the data folder. You could remove it after the
experiment.

The code also generate a log file under "logs" folder if one wants to analyze using matlab. You could
remove it after the experiment.

## Train Retain Model



In [3]:
%run main.py --dataset mnist --model mlp --dataroot=data/MNIST/ --filters 1.0 --lr 0.001 \
--resume checkpoints/standard_model_for_pilot.pt --disable-bn \
--weight-decay 0.1 --batch-size 128 --epochs 31 \
--forget-class 0,1,2,3,4,5 --num-to-forget 300 --seed 3

Checkpoint name: mnist_mlp_1_0_forget_[0, 1, 2, 3, 4, 5]_num_300_lr_0_001_bs_128_ls_ce_wd_0_1_seed_3
[Logging in mnist_mlp_1_0_forget_[0, 1, 2, 3, 4, 5]_num_300_lr_0_001_bs_128_ls_ce_wd_0_1_seed_3_training]
confuse mode: False
split mode: None
Replacing indexes [ 8526 16694 15152 29352   141 36509 37843 16926 35321 26042 21345  5714
 47547 14437 39793 44980 45137  3897 35780 29999 33655 37420 40980 28992
   105 40544 41016 46885 46091 40786  5543  7014 20505 25936 16792  2457
  1600 13007  7760 15774 46362 33702 32877  2543 33599 32886 35244 29239
  8640 23665 45315  3038 22738  3554  4458 36789 11462  2191 23706 10872
  9290 32216 25081 12968 28129 20523  5261 16762 26382 19263 12728 37875
 36911  7326 21202 31998 26810 28700 36488 42838  3180 24261 25761 13600
 18425 29148 39305  4254  2025  4307 45980 28150 36331  8747 35060 28517
  8694 12015 38188 36605 26116 12256 18607 11847  1442 19196 18989 33194
 26487  5803   828 40152 11342 43196 20715 41802 14043  8865 20144 41373
 47285  